In [1]:
# importamos las librerías necesarias
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter
import matplotlib.pyplot as plt 
import seaborn as sns
np.random.seed(639)
from sklearn.preprocessing import StandardScaler
from scipy import stats
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
import spacy
import re
from unidecode import unidecode

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# creamos dataframe con los nombres de los juejos
df_names = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Sistema de Recomendacion\\item_names.parquet')
df_names.head()

,item_id,item_name
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer


In [3]:
len(df_names)

31220

In [4]:
# Renombre las columnas del dataframe
df_names.rename(columns= {'item_id' : 'id', 'item_name': 'app_name'}, inplace = True)

In [5]:
# creamos dataframe con las features de los juegos
df = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\steam_games.parquet')
df.head()

,id,app_name,title,price,developer,release_date,tag_1980s,tag_1990's,tag_2.5d,tag_2d,...,spec_single-player,spec_stats,spec_steam achievements,spec_steam cloud,spec_steam leaderboards,spec_steam trading cards,spec_steam turn notifications,spec_steam workshop,spec_steamvr collectibles,spec_valve anti-cheat enabled
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,4.99,Kotoshiro,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,643980,Ironbound,Ironbound,0.00,Secret Level SRL,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,670290,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,Poolians.com,2017-07-24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400,弹炸人2222,弹炸人2222,0.99,彼岸领域,2017-12-07,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,Battle Royale Trainer,Battle Royale Trainer,3.99,Trickjump Games Ltd,2018-01-04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_names = df[['id', 'app_name']]
df_names.head()

,id,app_name
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer


In [7]:
df_names.to_parquet('c:\\Users\\Sofia\\Desktop\\Sistema de Recomendacion\\item_names.parquet')

In [8]:
# eliminamos columnas y lo combinamos con el dataframe de nombres
df.drop(columns=['app_name', 'title'], inplace=True)
df = df_names.merge(df, how='left')
df.head()

,id,app_name,price,developer,release_date,tag_1980s,tag_1990's,tag_2.5d,tag_2d,tag_2d fighter,...,spec_single-player,spec_stats,spec_steam achievements,spec_steam cloud,spec_steam leaderboards,spec_steam trading cards,spec_steam turn notifications,spec_steam workshop,spec_steamvr collectibles,spec_valve anti-cheat enabled
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,2018-01-04,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,643980,Ironbound,0.00,Secret Level SRL,2018-01-04,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,2017-07-24,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400,弹炸人2222,0.99,彼岸领域,2017-12-07,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,2018-01-04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# creamos la columna año y eliminamos la columna release_date
df['Year'] = df['release_date'].dt.year
df.drop(columns='release_date', inplace= True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28899 entries, 0 to 28898
Columns: 495 entries, id to Year
dtypes: float64(1), int32(492), object(2)
memory usage: 54.9+ MB


In [28]:
# importamos en un dataframe otro archivo con features de item
df_features = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Sistema de Recomendacion\\item_item_features.parquet')
df_features.rename(columns={'item_id': 'id'}, inplace= True)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10978 entries, 0 to 10977
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                10978 non-null  int32
 1   playtime_forever  10978 non-null  int64
 2   playtime_2weeks   10978 non-null  int64
dtypes: int32(1), int64(2)
memory usage: 214.5 KB


In [29]:
df_features.head()

,id,playtime_forever,playtime_2weeks
0,10,17107858,46656
1,20,960524,14117
2,30,756375,14042
3,40,154424,38
4,50,726545,2485


In [30]:
# lo combinamos con el dataframe original
df = df.merge(df_features, how='left')
df.head()

,id,app_name,price,developer,tag_1980s,tag_1990's,tag_2.5d,tag_2d,tag_2d fighter,tag_3d platformer,...,spec_steam workshop,spec_steamvr collectibles,spec_valve anti-cheat enabled,Year,rating,recommend,positive_ratio,name_feature,playtime_forever,playtime_2weeks
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,lose summoner kitty,NaN,NaN
1,643980,Ironbound,0.00,Secret Level SRL,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,ironbound,NaN,NaN
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,0,0,0,0,0,0,...,0,0,0,2017,NaN,NaN,NaN,real pool 3d poolians,NaN,NaN
3,767400,弹炸人2222,0.99,彼岸领域,0,0,0,0,0,0,...,0,0,0,2017,NaN,NaN,NaN,,NaN,NaN
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,battle royale trainer,NaN,NaN


In [31]:
df.duplicated().sum()               # vemos que no hay duplicados

np.int64(0)

In [32]:
# importamos los datos de análisis de sentimiento
df_sent = pd.read_parquet("c:\\Users\\Sofia\\Desktop\\sentiment_analysis_2.parquet")
df_sent.head()

,user_id,item_id,sentiment_analysis_2,recommend,Año
0,76561197970982479,1250,2,1,2011
1,76561197970982479,22200,2,1,2011
2,76561197970982479,43110,2,1,2011
3,js41637,251610,2,1,2014
4,js41637,227300,2,1,2013


In [33]:
# generamos las features rating y positive_ratio
df_sent.drop(columns=['user_id', 'Año'], inplace=True)
df_sent['positive_ratio'] = np.where(df_sent['sentiment_analysis_2'] == 2, 1, 0)
df_sent['cant_reviews'] = np.where(df_sent['item_id'] != 0, 1, 0)
df_sent_grouped = df_sent.groupby('item_id').agg(sum)
df_sent_grouped.rename(columns={'sentiment_analysis_2': 'rating'}, inplace=True)
df_sent_grouped.reset_index(inplace=True)
df_sent_grouped.rename(columns={'item_id': 'id'}, inplace=True)

In [34]:
df_sent_grouped['positive_ratio'] = df_sent_grouped['positive_ratio'] / df_sent_grouped['cant_reviews']
df_sent_grouped.drop(columns='cant_reviews', inplace= True)

In [35]:
df_sent_grouped.head()

,id,rating,recommend,positive_ratio
0,10,90,55,0.678571
1,20,25,11,0.588235
2,30,7,3,0.750000
3,40,2,1,1.000000
4,50,8,3,1.000000


In [36]:
# combinamos con el dataframe de features que estamos construyendo
df = df.merge(df_sent_grouped, how='left')
df.head()

,id,app_name,price,developer,tag_1980s,tag_1990's,tag_2.5d,tag_2d,tag_2d fighter,tag_3d platformer,...,spec_steam workshop,spec_steamvr collectibles,spec_valve anti-cheat enabled,Year,rating,recommend,positive_ratio,name_feature,playtime_forever,playtime_2weeks
0,761140,Lost Summoner Kitty,4.99,Kotoshiro,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,lose summoner kitty,NaN,NaN
1,643980,Ironbound,0.00,Secret Level SRL,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,ironbound,NaN,NaN
2,670290,Real Pool 3D - Poolians,0.00,Poolians.com,0,0,0,0,0,0,...,0,0,0,2017,NaN,NaN,NaN,real pool 3d poolians,NaN,NaN
3,767400,弹炸人2222,0.99,彼岸领域,0,0,0,0,0,0,...,0,0,0,2017,NaN,NaN,NaN,,NaN,NaN
4,772540,Battle Royale Trainer,3.99,Trickjump Games Ltd,0,0,0,0,0,0,...,0,0,0,2018,NaN,NaN,NaN,battle royale trainer,NaN,NaN


In [37]:
# En esta sección se pueden generar features con las palabras de los nombres 
stopwords = nltk.corpus.stopwords.words('english')

def clean_text(name):
    if pd.isna(name) or name is None:
        return name
    else:
        # limpieza
        name = str(name)
        name = name.lower()
        name= name.replace('\n', '')
        name = unidecode(name)
        name = re.sub(r'https?://\S+|www\.\S+', '', name)

        # tokenización
        name = nltk.tokenize.RegexpTokenizer("[\w]+").tokenize(name)

        # quitar stopwords
        name = [word.strip() for word in name if word not in stopwords]
        return name

nlp = spacy.load('en_core_web_sm')          # cargamos nlp en inglés

# definimos una función para cambiar la función posicional de las palabras a una expresión de una sola letra
def get_wordnet_pos(pos):
    if pos == 'VERB':
        return 'v'
    elif pos == 'ADJ':
        return 'a'
    elif pos == 'ADV':
        return 'r'
    elif pos == 'NOUN':
        return 'n'
    else:
        return 'n'

# definimos una función para lemmatizar el texto de cada review
def lemmatize_text(name):
    if name is None:
        return name

    lemmatizer = WordNetLemmatizer()

    # Obtener posiciones gramaticales en un conjunto
    pos_set = {'VERB', 'ADJ', 'ADV', 'NOUN'}

    # Lematizar en lotes usando spaCy
    doc = nlp(' '.join(name))

    lemmatized_words = [lemmatizer.lemmatize(token.text, get_wordnet_pos(token.pos_)) for token in doc if token.pos_ in pos_set]

    return lemmatized_words


In [38]:
df['name_feature'] = df['app_name'].apply(clean_text)

In [39]:
df['name_feature'] = df['name_feature'].apply(lemmatize_text)

In [40]:
df['name_feature'] = df['name_feature'].apply(lambda x: ' '.join(x) if x else '')
dummy_df = pd.get_dummies(df['name_feature'].str.split(expand=True).stack(), prefix='game')
dummy_df.head()

game_08s  game_0one  game_100days  game_10a  game_10c  game_10th  \
0 0     False      False         False     False     False      False   
  1     False      False         False     False     False      False   
  2     False      False         False     False     False      False   
1 0     False      False         False     False     False      False   
2 0     False      False         False     False     False      False   

     game_111th  game_11th  game_12ga  game_14xx  ...  game_zorbit  game_zork  \
0 0       False      False      False      False  ...        False      False   
  1       False      False      False      False  ...        False      False   
  2       False      False      False      False  ...        False      False   
1 0       False      False      False      False  ...        False      False   
2 0       False      False      False      False  ...        False      False   

     game_zotrix  game_zpeciation  game_zu  game_zubmariner  game_zulu  \
0 0        False            False    False            False      False   
  1        False            False    False            False      False   
  2        False            False    False            False      False   
1 0        False            False    False            False      False   
2 0        False            False    False            False      False   

     game_zuruck  game_zyd  game_zzzz  
0 0        False     False      False  
  1        False     False      False  
  2        False     False      False  
1 0        False     False      False  
2 0        False     False      False  

[5 rows x 11164 columns]

In [ ]:
# no generaremos las features de palabras ya que son mucha cantidad de columnas y computacionalmente será muy costoso para nuestra utilidad

In [41]:
# eliminamos las columnas 'id' y 'app_name' ya que no son features
df_items = df.drop(columns='id')
df_items = df_items.drop(columns='app_name')

In [27]:
print(df_items.columns)


Index(['price', 'developer', 'tag_1980s', 'tag_1990's', 'tag_2.5d', 'tag_2d',
       'tag_2d fighter', 'tag_3d platformer', 'tag_3d vision',
       'tag_4 player local',
       ...
       'spec_steam trading cards', 'spec_steam turn notifications',
       'spec_steam workshop', 'spec_steamvr collectibles',
       'spec_valve anti-cheat enabled', 'Year', 'rating', 'recommend',
       'positive_ratio', 'name_feature'],
      dtype='object', length=497)


In [42]:
df_items['playtime_forever'].fillna(0, inplace=True)                                # imputamos nulos con cero
df_items['playtime_2weeks'].fillna(0, inplace=True)                                 # imputamos nulos con cero
df_items['recommend'].fillna(np.nanmedian(df_items['recommend']), inplace=True)     # imputamos nulos con la mediana
df_items['rating'].fillna(np.nanmedian(df_items['rating']), inplace=True)           # imputamos nulos con la mediana

In [43]:
scaler = StandardScaler()       # escalamos con z-score
df_items['playtime_forever'] = scaler.fit_transform(df_items['playtime_forever'].array.reshape(-1,1))
scaler = StandardScaler()       
df_items['playtime_2weeks'] = scaler.fit_transform(df_items['playtime_2weeks'].array.reshape(-1,1))

In [44]:
scaler = StandardScaler()       
df_items['recommend'] = scaler.fit_transform(df_items['recommend'].array.reshape(-1,1))
scaler = StandardScaler()      
df_items['rating'] = scaler.fit_transform(df_items['rating'].array.reshape(-1,1))

In [45]:
df_items.drop(columns='developer', inplace=True)        # eliminamos la columna developer que tampoco es feature

In [46]:
# se ve como queda nuestro dataframe con features de items
df_items.head()

,price,tag_1980s,tag_1990's,tag_2.5d,tag_2d,tag_2d fighter,tag_3d platformer,tag_3d vision,tag_4 player local,tag_4x,...,spec_steam workshop,spec_steamvr collectibles,spec_valve anti-cheat enabled,Year,rating,recommend,positive_ratio,name_feature,playtime_forever,playtime_2weeks
0,4.99,0,0,0,0,0,0,0,0,0,...,0,0,0,2018,-0.040726,-0.038933,NaN,lose summoner kitty,-0.026467,-0.026325
1,0.00,0,0,0,0,0,0,0,0,0,...,0,0,0,2018,-0.040726,-0.038933,NaN,ironbound,-0.026467,-0.026325
2,0.00,0,0,0,0,0,0,0,0,0,...,0,0,0,2017,-0.040726,-0.038933,NaN,real pool 3d poolians,-0.026467,-0.026325
3,0.99,0,0,0,0,0,0,0,0,0,...,0,0,0,2017,-0.040726,-0.038933,NaN,,-0.026467,-0.026325
4,3.99,0,0,0,0,0,0,0,0,0,...,0,0,0,2018,-0.040726,-0.038933,NaN,battle royale trainer,-0.026467,-0.026325


In [47]:
df_items.isna().sum()

price                   0
tag_1980s               0
tag_1990's              0
tag_2.5d                0
tag_2d                  0
                    ...  
recommend               0
positive_ratio      25804
name_feature            0
playtime_forever        0
playtime_2weeks         0
Length: 498, dtype: int64

In [48]:
df_items['price'].fillna(np.nanmedian(df_items['price']), inplace=True)     # completamos nulos de precio con mediana

In [49]:
df_items['Year'].fillna(stats.mode(df_items['Year'])[0], inplace=True)      # completamos nulos de año con la moda

In [50]:
df_items.columns[df_items.isnull().any()].tolist()                          # obtenemos la lista de columnas con nulos

['positive_ratio']

In [51]:
df_items.fillna(0, inplace=True)                    # completamos las features que faltan con 0

In [52]:
len(df_items)

28899

In [53]:
df_items.to_parquet('c:\\Users\\Sofia\\Desktop\\Sistema de Recomendacion\\item_features_complete.parquet')

In [54]:
pd.set_option('display.max_columns', None)

In [55]:
#definimos una función para obtener items similares según cosine similarity
def item_item_recom(item_id, n_recom=5, umbral= 0.9999999):
    
    try: 
        item_id = int(item_id)
        indice = df_names[df_names['id'] == item_id].index[0]       # verificamos que el item_id exista
    except Exception as e:
        print(f'Error: {e}')
        return print('Ingrese un Id de producto válido')            # retornamos mensaje en caso de no existir el id
    
    similaridades = {}                                              # iniciamos diccionario de similaridades
    contador = 0                                                    # contador para realizar corte por umbral
    for i in range(len(df_items)):                      # recorremos el dataframe
        if i != indice:                                 # no tomamos el indice del juego de entrada
            sim = cosine_similarity(df_items.iloc[indice,:-4].values.reshape(1,-1), df_items.iloc[i,:-4].values.reshape(1,-1))[0][0]
            similaridades[i] = sim                      # calclualmos la similaridad y la guardamos en el diccionario
            if sim > umbral:                            # verificamos si la similaridad es mayor al umbral predeterimnado
                contador += 1                           # si es mayor, sumamos 1 al contador
            if contador > n_recom:                      # si se supera la cantidad de recomendaciones del contador, detenemos la búsqueda
                break
    
    similaridades_sorted = sorted(similaridades.items(), key= itemgetter(1), reverse=True)      # ordenamos las similaridades por valores

    items_recomendados = []                             # creamos diccionario vacío para acumular los juegos recomendados

    for i in range(n_recom):                            # nos quedamos con los indices de los n_recom primeros juegos
        items_recomendados.append(similaridades_sorted[i][0])
        
    return df_names.loc[items_recomendados,'app_name']  # retornamos dataframe de recomendados

In [56]:
item_item_recom(730)

28646    Team Fortress 2
47           Garry's Mod
15057           Unturned
28048           Terraria
26503               Rust
Name: app_name, dtype: object

In [57]:
df_names.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28899 entries, 0 to 28898
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        28899 non-null  int32 
 1   app_name  28899 non-null  object
dtypes: int32(1), object(1)
memory usage: 1.6+ MB


In [58]:
df_names.iloc[[1023, 10036, 11608, 5010, 6288, 3153],:]

,id,app_name
1023,215243,Sleeping Dogs: The Red Envelope Pack
10036,643910,Ashes of the Singularity: Escalation - Co-Op M...
11608,659330,Rubber Toys
5010,434250,Masked Shooters 2
6288,500160,Alteric - Original Soundtrack
3153,327470,There Came an Echo: Original Soundtrack
